People-to-People correlation 協同過濾       

1. 首先，找到User1 喜歡的電影；     
2. 找出與User1具有相同的電影興趣愛好的人群；            
3. 找出該人群喜歡的其他電影；           
4. 將這些電影推送給User1。          

data        
數據包含138,000 個用戶對27,000 個電影的評價，評價分為0-5            
1. ranking: ml-latest-small(1MB)        
2. movies： ml-latest(234.2MB)      

In [28]:
# pip install pandas
import pandas as pd                         

ratings = pd.read_csv('./ml-latest/ratings.csv')    
# print (ratings.head(20)) 
movies = pd.read_csv('./ml-latest/movies.csv') 
# print(movies.head(5))

In [29]:
data = pd.merge(movies, ratings, on = 'movieId')
# print(data.head(5))
data[['userId','rating','movieId','title']].sort_values('userId').to_csv('./data.csv',index=False)
data = pd.read_csv('data.csv')
# print(data.head(5))


In [30]:
# 字典來表示每位用戶評論的電影和評分        

# 逐行读取刚刚合并并保存的数据集
content = []
with open('./data.csv',"r",encoding="utf-8") as fp:
    content = fp.readlines()

# 将用户、评分、和电影写入字典data
data = {}
for line in content[1:100]:
    line = line.strip().split(',')
    #如果字典中没有某位用户，则使用用户ID来创建这位用户
    if not line[0] in data.keys():
        data[line[0]] = {line[3]:line[1]}
    #否则直接添加以该用户ID为key字典中
    else:
        data[line[0]][line[3]] = line[1]


data 

{'1': {'Hollow Man (2000)': '2.0',
  'RoboCop 2 (1990)': '2.5',
  "Weekend at Bernie's (1989)": '1.5',
  '"Running Man': '3.5',
  'Spawn (1997)': '1.5',
  'Falling Down (1993)': '4.0',
  'Event Horizon (1997)': '2.5',
  'Do the Right Thing (1989)': '4.5',
  'Stigmata (1999)': '3.0',
  'Kalifornia (1993)': '3.5',
  'Nurse Betty (2000)': '3.5',
  '¡Three Amigos! (1986)': '4.0',
  'Better Off Dead... (1985)': '4.5',
  'Weird Science (1985)': '4.5',
  'Three Colors: Blue (Trois couleurs: Bleu) (1993)': '3.5',
  'Waiting for Guffman (1996)': '4.5'},
 '2': {'"Room with a View': '4.5',
  'Broadcast News (1987)': '4.5',
  'Arlington Road (1999)': '3.5',
  'Manhattan (1979)': '3.0',
  'Risky Business (1983)': '3.5',
  'Little Shop of Horrors (1986)': '4.0',
  'Escape from L.A. (1996)': '3.5',
  'Hackers (1995)': '3.5',
  'Driving Miss Daisy (1989)': '4.0',
  'L.A. Story (1991)': '3.5',
  'American Graffiti (1973)': '4.0',
  '"Sex': '3.5',
  '"Big Chill': '4.0',
  'Stripes (1981)': '3.0',
  'Har

In [31]:
# 計算任兩位用戶間的相似度
# 找共同評論過的電影, 計算兩者之間的歐式距離(相似度)。

from math import *

def Euclidean(user1,user2):
    
    user1_data = data[user1]                     #取出两位用户评论过的电影和评分
    user2_data = data[user2]
    distance = 0
    
    for key in user1_data.keys():               #找到两位用户都评论过的电影，并计算欧式距离
        if key in user2_data.keys():
            #注意，distance越大表示两者越相似
            distance += pow(float(user1_data[key])-float(user2_data[key]),2)
 
    return 1/(1+sqrt(distance))                 #这里返回值越小，相似度越大


#计算某个用户与其他用户的相似度
def top10_simliar(userID):
    res = []
    for userid in data.keys():
        
        if not userid == userID:                #排除与自己计算相似度
            simliar = Euclidean(userID, userid)
            res.append((userid, simliar))
    res.sort(key=lambda val:val[1])
    
    return res[:4]
 
RES = top10_simliar('1')

print(RES)
# 用户之间相似度结果：0表示两位的影评几乎一样，1表示没有共同的影评



[('2', 1.0), ('3', 1.0), ('4', 1.0)]


In [32]:
# 根據相似度來推薦用戶：

def recommend(user):
    
    top_sim_user = top10_simliar(user)[0][0]                    #相似度最高的用户
    items = data[top_sim_user]                                  #相似度最高的用户的观影记录
    recommendations = []
    
    for item in items.keys():                                   #筛选出该用户未观看的电影并添加到列表中
        if item not in data[user].keys():
            recommendations.append((item,items[item]))
    recommendations.sort(key=lambda val:val[1], reverse=True)    #按照评分排序
    
    return recommendations[:10]                                 #返回评分最高的10部电影
 
Recommendations = recommend('1')
print(Recommendations)


[('"Room with a View', '4.5'), ('Broadcast News (1987)', '4.5'), ('Little Shop of Horrors (1986)', '4.0'), ('Driving Miss Daisy (1989)', '4.0'), ('American Graffiti (1973)', '4.0'), ('"Big Chill', '4.0'), ('Arlington Road (1999)', '3.5'), ('Risky Business (1983)', '3.5'), ('Escape from L.A. (1996)', '3.5'), ('Hackers (1995)', '3.5')]
